In [2]:
# run_test.py
import torch
from standard_rfm import StandardLaplaceRFM

# Tạo dữ liệu mẫu đơn giản
torch.manual_seed(0)
X_train = torch.randn(100, 10)
y_train = torch.randn(100, 1)
X_test = torch.randn(20, 10)
y_test = torch.randn(20, 1)

# Tạo mô hình
device = torch.device('cpu')  # Chuyển sang 'cuda' nếu có GPU
model = StandardLaplaceRFM(bandwidth=1.0, device=device)

# Huấn luyện mô hình
model.fit((X_train, y_train), (X_test, y_test), verbose=True)

# Dự đoán
preds = model.predict(X_test)
print(f"Hình dạng dự đoán: {preds.shape}")


ModuleNotFoundError: No module named 'standard_rfm'

# Kaggle


In [ ]:


import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from rfm import LaplaceRFM, GeneralizedLaplaceRFM, GaussRFM, NTKModel
import torch.nn.functional as F
import logging

# Cấu hình logging: ghi log vào file và in ra console
logger = logging.getLogger()
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(message)s')

# File handler: ghi log vào output.txt
file_handler = logging.FileHandler("output.txt")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Console handler: in log ra console
console_handler = logging.StreamHandler()
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

# Hàm chuyển đổi batch: chuyển đổi nhãn thành one-hot encoding với 10 lớp
def one_hot_collate(batch):
    images, labels = zip(*batch)
    images = torch.stack(images, 0)
    labels = torch.tensor(labels)
    # Chuyển đổi nhãn sang one-hot (float)
    labels = F.one_hot(labels, num_classes=10).float()
    return images, labels

# Thiết lập thiết bị và bộ nhớ GPU (nếu có)
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    DEV_MEM_GB = torch.cuda.get_device_properties(DEVICE).total_memory // 1024**3 - 1 
else:
    DEVICE = torch.device("cpu")
    DEV_MEM_GB = 8


transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])


full_train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)


subset_size = 10000  
train_subset, _ = random_split(full_train_dataset, [subset_size, len(full_train_dataset) - subset_size])


batch_size = 16
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, collate_fn=one_hot_collate)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=one_hot_collate)


default_p_batch_size = 8


laplace_model = LaplaceRFM(bandwidth=1., device=DEVICE, mem_gb=DEV_MEM_GB, diag=False, p_batch_size=default_p_batch_size)
generalized_model = GeneralizedLaplaceRFM(bandwidth=1., device=DEVICE, mem_gb=DEV_MEM_GB, diag=False, p_batch_size=default_p_batch_size)
gauss_model = GaussRFM(bandwidth=1., device=DEVICE, mem_gb=DEV_MEM_GB, diag=False, p_batch_size=default_p_batch_size)
ntk_model = NTKModel(device=DEVICE, mem_gb=DEV_MEM_GB, diag=False, p_batch_size=default_p_batch_size)

c:\Users\hoang\anaconda3\envs\clean_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
laplace_model.fit(
    train_loader, 
    test_loader, 
    loader=True, 
    iters=3,
    classification=True,  # Use the proper parameter name (classification instead of classif)
    total_points_to_sample=subset_size,
    M_batch_size=batch_size,
    p_batch_size=default_p_batch_size
 )

Loaders provided
Round 0, Train Acc: 100.00%, Test Acc: 92.36%
Round 0, Test MSE: 0.0617
Sampling AGOP on 10016 total points


100%|██████████| 625/625 [04:31<00:00,  2.30it/s]


Round 1, Train Acc: 100.00%, Test Acc: 93.49%
Round 1, Test MSE: 0.0742
Sampling AGOP on 10016 total points


100%|██████████| 625/625 [04:18<00:00,  2.42it/s]


Round 2, Train Acc: 100.00%, Test Acc: 92.35%
Round 2, Test MSE: 0.0650
Sampling AGOP on 10016 total points


100%|██████████| 625/625 [04:18<00:00,  2.41it/s]


Final MSE: 0.0531
Final Test Acc: 91.00%


tensor(0.0531)